In [1]:
from IPython.display import clear_output
import requests
import csv
import os

from PyPDF2 import *

In [2]:
competitions = [('Serie_A', '142'),
                ('Serie_B', '242'),
                ('Serie_C', '342'),
                ('Serie_D', '542'),
                ('CdB', '424')]

for competition in competitions:
    name, cod = competition
    os.mkdir(name)
    os.chdir(name)
    for year in range(2013, 2023):
        os.mkdir(f'{year}')
        os.chdir(f'{year}')
        os.mkdir('PDFs')
        os.mkdir('CSVs')
        os.chdir('..')
    
    os.chdir('..')

In [3]:
errors = []
for competition in competitions:
    competition, cod = competition
    for year in range(2013, 2023):
        print(f'Iniciando ano de {year} para {competition.replace("_", " ")}')
        cont = 0
        for game in range(1, 1000):
            if cont == 20:
                break
                
            try:
                name = f'{competition}/{year}/PDFs/{str(game).zfill(3)}.pdf'
                pdf = requests.get(f'https://conteudo.cbf.com.br/sumulas/{year}/{cod}{game}se.pdf').content
                if b'File or directory not found' in pdf:
                    cont += 1
                    continue

                with open(name, 'wb') as f:
                    f.write(pdf)

                reader = PdfReader(name)
                doc = []
                for i in range(len(reader.pages)):
                    page = reader.pages[i]
                    doc += page.extract_text().split('\n')

                for i in range(len(doc)):
                    doc[i] = [doc[i]]

                name = name.replace('PDFs', 'CSVs')
                name = name.replace('pdf', 'csv')
                with open(name, 'w') as f:
                    write = csv.writer(f)
                    write.writerows(doc)

            except:
                errors.append(name)

        print(f'Finalizado ano de {year} para {competition.replace("_", " ")}')
    
    clear_output()

len(errors)

2

In [4]:
errors

['Serie_B/2020/PDFs/135.pdf', 'Serie_D/2020/PDFs/042.pdf']